# Breast Cancer Prediction Menggunakan Binary Classification (Neural Network)

Anggota :
1.   Johana Paskalina Sihotang (Accuracy & Model Loss Plot , Testing Program)
2.   Rodiatul Husna Br Sitepu (Data Wrangling, Data Validation & Model Creation)
3.   A. Nurcahaya Tampubolon  (Data Cleansing, Data Training & Evaluation)
4.   Yessica Angelita Situmorang (Import Library & Load Dataset)


## Import Library

In [ ]:
import pandas as pd # digunakan untuk manipulasi dan analisis data
import numpy as np # digunakan untuk komputasi numerik dalam python
import tensorflow as tf # digunakan untuk membangun, melatih dan menerapkan model pembelajaran mesin
from tensorflow import keras # digunakan untuk membangun dan melatih model menggunakan jaringan neural networks
from matplotlib import pyplot as plt # digunakan untuk plotting data
%matplotlib inline

## Load Dataset

Dataset Download: [Breast_Cancer_Prediction.csv](https://github.com/johanasht/Prediksi-Kanker-Payudara/blob/main/breast_cancer_prediction.csv)

In [ ]:
# Upload data
from google.colab import files
uploaded = files.upload()

In [ ]:
# membaca data
df = pd.read_csv('breast_cancer_prediction.csv')
df

In [ ]:
# membaca data
df.info()

## Data Preprocessing

### Data Wrangling

In [ ]:
# menampilkan kolom dataset
df.columns

In [ ]:
# memilih kolom yang diperlukan untuk proses pengujian
df = df[['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]

In [ ]:
# menampilkan kembali dataset yang sudah difilter kolomnya
df

In [ ]:
# menampikan isi dari kolom diagnosis
df['diagnosis'].unique()

Disini **'M'** digunakan untuk menunjukkan status ***malignant*** yang artinya kanker tersebut bersifat ganas sedangkan **'B'** menunjukkan status ***benign*** yang artinya kanker tersebut bersifat jinak

### Data Cleansing

In [ ]:
# melihat seluruh kolom beserta dengan type data nya, jumlah, unique values dan missing value nya
# diurutkan secara descending berdasarkan missing values
missing = []
unique = []
types = []
variables = []
count = []

for item in df.columns:
    variables.append(item)
    missing.append(df[item].isnull().sum())
    unique.append(df[item].nunique())
    types.append(df[item].dtypes)
    count.append(len(df[item]))

output = pd.DataFrame({
    'variable': variables,
    'dtype': types,
    'count': count,
    'unique': unique,
    'missing': missing,

})
output.sort_values("missing",ascending=False).reset_index(drop=True)

In [ ]:
# cek apakah ada data yang duplikat
df.duplicated().sum()

In [ ]:
# menggunakan boxplot untuk melihat apakah ada outliers di tiap kolom dari dataset
df.boxplot(column=list(df.select_dtypes(include=[np.number]).columns))
plt.xticks(np.arange(1, len(df.columns) + 1), df.columns, rotation=45, ha='right')
plt.show()

In [ ]:
# melihat persebaran data tiap kolom
df.describe()

In [ ]:
# function untuk mengecek apakah terdapat outliers dalam data menggunakan IQR
def outliers(df, feature):
  q1 = df[feature].quantile(0.25)
  q3 = df[feature].quantile(0.75)
  iqr = q3 - q1

  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr

  ls = df.index[ (df[feature] < lower_bound) | (df[feature] > upper_bound) ]

  return ls

In [ ]:
df.columns

In [ ]:
# membuat sebuah list dengan nama index_list yang akan menampung index dari outliers dari semua feature yang terdapat pada data
index_list = []
for feature in ['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']:
               index_list.extend(outliers(df, feature))

index_list

In [ ]:
# function untuk menghapus data yang termasuk outliers
def remove(df, ls):
  ls = sorted(set(ls))
  df = df.drop(ls)
  return df

In [ ]:
# variable df_cleaned akan memanggil function remove dan menyimpan data yang outliers nya sudah dihapus
df = remove(df, index_list)
# mengecek jumlah baris dan kolom setelah outliers sudah dihapus
df.shape

## Data Validation

In [ ]:
# kita gunakan replace untuk mengganti M menjadi 1 dan B menjadi 0 agar
# nantinya dapat digunakan untuk proses binary classification
df['diagnosis'].replace(['M', 'B'], [1, 0], inplace=True)

In [ ]:
# menampilkan info tipe data dan frekuensi tiap kolom
df.info()

In [ ]:
df.columns

In [ ]:
# membuat variabel X yang akan menyimpan variabel independen dan variabel y
# yang akan mwnyimpan variabel dependen
X = df[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']]
y = df['diagnosis']

# Model Creation


In [ ]:
from sklearn import preprocessing # menyediakan fungsi preprocessing data

# melakukan penskalaan fitur dengan metode Min-Max Scaling. Metode ini
# mengubah fitur-fitur menjadi rentang yang lebih kecil, yaitu dari 0 hingga 1.
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

In [ ]:
# membagi dataset menjadi training set dan test set dimana
# 70% data dipakai untuk training dan 30% untuk testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  X_scale, y, test_size=0.3
)

In [ ]:
# Mengubah tipe data variabel y_train dan y_test menjadi float
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

## Data Training

In [ ]:
# Membangun model neural network yaitu binary classification
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
  Dense(1024, activation='relu', input_shape=[10]), # 10 input, 1024 neuron
  Dense(1024, activation='relu'), # 1024 neuron
  Dense(1, activation='sigmoid'), # 1 output
])

# Fungsi aktivasi sigmoid digunakan untuk masalah klasifikasi biner,
# di mana output yang dihasilkan berada dalam rentang antara 0 dan 1

In [ ]:
# Konfigurasi model
model.compile(
  optimizer='adam', # optimizer Adam digunakan untuk optimasi
  loss='binary_crossentropy', # binary cross entropy untuk binary classification
  metrics=['binary_accuracy'], # mengukur persentase jumlah prediksi benar terhadap total data
)

In [ ]:
# Melatih model
h = model.fit(X_train, y_train, epochs=30)

In [ ]:
# Mengevaluasi model yang sudah dibuat
model.evaluate(X_test, y_test)

Dari hasil evaluasi, didapatkan nilai akurasi sebesar 95,36% persen yang menunjukkan bahawa model memiliki tingkat akurasi yang cukup tinggi dan nilai loss sebesar 0.1797 yang rendah yang menandakan bahwa model yang dibuat ini mempunyai hasil yang baik.

In [ ]:
# Menampilkan akurasi model dalam bentuk plot
plt.plot(h.history['binary_accuracy'])
plt.title('Model Accuracy Plot')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='lower right')
plt.show()

Penjelasan Plot : Dalam grafik diatas, akurasi model memiliki nilai [ 0.9536423683166504] pada Epoch ke 30 dari yang awalnya kurang dari (<) [0.775] pada Epoch 0. Ini menunjukkan bahwa model Anda memiliki akurasi sekitar 95.364 %, yang berarti model berhasil memprediksi dengan benar sekitar 95.364 % dari semua sampel yang dievaluasi.


In [ ]:
# Menampilkan model loss dalam bentuk plot
plt.plot(h.history['loss'])
plt.title('Model Loss Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

Penjelasan Plot : Dalam grafik diatas, model loss memiliki nilai [0.17974278330802917] pada Epoch ke 30 dari yang awalnya lebih dari (>) [0.5] pada Epoch 0. Nilai ini menunjukkan seberapa besar perbedaan antara prediksi model dengan nilai yang sebenarnya. Semakin rendah nilai loss, semakin baik performa model yang dibuat. Jadi, dalam contoh ini, model memiliki loss yang rendah, yang mengindikasikan bahwa prediksi model secara bertahap memperbaiki prediksinya seiring berjalannya pelatihan.

#Testing Program


In [ ]:
# Data pengujian
x_test = np.array([[17.99, 10.38, 122.80, 1001.0, 0.11840, 0.27760, 0.30010, 0.14710, 0.2419, 0.07871],
                   [16.6, 12.7, 120.0, 1000.0, 0.1, 0.3, 0.3, 0.17, 0.2134, 0.063],
                   [7.76, 24.54, 47.92, 181.0, 0.05263, 0.04362, 0.00000, 0.00000, 0.1587, 0.05884]])

# Melakukan penskalaan pada data pengujian
x_test_scaled = min_max_scaler.transform(x_test)

# Melakukan prediksi menggunakan model
predictions = model.predict(x_test_scaled)

# Mengubah nilai prediksi menjadi label klasifikasi (M = Malignant (Ganas), B = Benign (Jinak))
class_labels = ['B', 'M']

# Menampilkan hasil prediksi
for i in range(len(predictions)):
    predicted_label = class_labels[int(np.round(predictions[i]))]
    print("Hasil Prediksi {}:".format(i+1), predicted_label)

#Conclusion


Kesimpulan:

Program breast cancer detection yang telah dibuat menggunakan binary classification (neural networks) mampu mencapai tingkat akurasi antara 94% hingga 96% dengan tingkat kehilangan (model loss) sebesar 9% hingga 18%. Hal ini menunjukkan bahwa program tersebut memiliki performa yang baik dalam mengklasifikasikan apakah pasien mengidap kanker payudara atau tidak.

Dengan tingkat akurasi yang tinggi, yaitu antara 94% hingga 96%, program dapat memberikan prediksi yang akurat dalam mengidentifikasi pasien yang terkena kanker payudara. Tingkat akurasi ini menunjukkan bahwa sebagian besar pasien dengan kanker payudara dapat terdeteksi dengan benar oleh program.

Selain itu, tingkat kehilangan (model loss) program yang berkisar antara 9% hingga 18% juga cukup rendah. Hal ini menunjukkan bahwa program memiliki kemampuan yang baik dalam meminimalkan kesalahan atau ketidakcocokan antara prediksi dan nilai sebenarnya.

Dengan tingkat akurasi yang tinggi dan tingkat kehilangan yang rendah, program breast cancer detection ini dapat digunakan sebagai alat yang berguna dalam membantu diagnosis dini kanker payudara. Dengan mendeteksi kanker payudara pada tahap awal, pasien dapat segera mendapatkan perawatan yang tepat, meningkatkan peluang kesembuhan, dan mengurangi dampak yang ditimbulkan oleh penyakit ini.

Namun, perlu diingat bahwa hasil prediksi dari program ini masih memerlukan verifikasi lebih lanjut oleh tenaga medis yang berkualifikasi. Program ini sebaiknya digunakan sebagai alat pendukung dan bukan sebagai pengganti diagnosa dan perawatan medis yang profesional.

Dengan demikian, program breast cancer detection dengan tingkat akurasi 94% hingga 96% dan tingkat kehilangan 9% hingga 18% memiliki potensi besar dalam membantu deteksi dini kanker payudara dan meningkatkan kesempatan penyembuhan pasien.